# Notebook #4 - Pathfinder Timing and Profiling 

Initial steps: We must import specific environment variables to point to the user's notebook code director and to the Lucata tools.

In [1]:
import os
from IPython.display import Code, HTML
os.environ["USER_NOTEBOOK_CODE"]=os.environ["HOME"]+"/pearc-tutorial/code"
os.environ["PATH"]=os.pathsep.join(["/tools/emu/pathfinder-sw/22.02/bin",os.environ["PATH"]])
#os.environ["PATH"]=os.pathsep.join(["/tools/emu/pathfinder-sw/21.06/bin",os.environ["PATH"]])

This notebook goes along with the [Lucata profiling and timing slides](https://github.com/gt-crnch-rg/pearc-tutorial-2021/blob/main/slides/lucata_tutorial/03_Lucata_Pathfinder_Tutorial_Performance_Profiling.pdf), so please follow along with the slides for a supplemental resource. 

## Lucata Timing Hooks

The Lucata toolchain includes a simulation profiler called `emusim_profile`. Running the profiler on your entire program can take a long time, so the toolchain provides timing hooks to specify regions you want data for. Here we annotate SAXPY by placing timing hooks around the main computational kernel.

In [2]:
Code('timing-hooks-saxpy.c')

#include <stdio.h>
#include <stdlib.h>
#include <cilk/cilk.h>

#include "memoryweb.h"
#include <emu_c_utils/emu_c_utils.h>

void saxpy(long n, float a, float *x, float *y)
{
  for (long i = 0; i < n; i++)
    y[i] += a * x[i];
}

int main(int argc, char **argv)
{
  long num = atol(argv[1]); // number blocks
  long size = atol(argv[2]); // block size
  float aval = atof(argv[3]); // constant
  float **x = mw_malloc2d(num, size * sizeof(*x));
  float **y = mw_malloc2d(num, size * sizeof(*y));

  for (long j = 0; j < num; j++) {
    for (long i = 0; i < size; i++) {
      x[j][i] = j * size + i; y[j][i] = 0;
    }
  }

  hooks_region_begin("example");
  for (long i = 0; i < num; i++) {
    cilk_spawn_at (y[i]) saxpy(size, aval, x[i], y[i]);
  }
  cilk_sync;

  double time_ms = hooks_region_end();
  printf("time (ms) = %lf\n", time_ms);
}

### Profiling Output
We can then compile and profile the code. This will generate a separate folder "saxpy_profile" with HTML output files that you can then investigate in further detail. 

Note that this process might take a long time! For this reason it is important to only scope the region of interest in your code that you want to gather statistics for. Simulation should take under 3 minutes.

In [3]:
%%bash
FLAGS="-I/tools/lucata/pathfinder-sw/22.02/include/memoryweb/ -L/tools/lucata/pathfinder-sw/22.02/lib -lmemoryweb"
emu-cc -o timing-hooks-saxpy.mwx timing-hooks-saxpy.c -l emu_c_utils $FLAGS

In [4]:
%%bash
time emusim_profile saxpy_profile --total_nodes 2 -- timing-hooks-saxpy.mwx 8 128 5.0 

Generating profile in saxpy_profile/timing-hooks-saxpy
emusim.x  --total_nodes 2
timing-hooks-saxpy.mwx 8 128 5.0
Start untimed simulation with local date and time= Mon Sep 19 15:40:17 2022

End untimed simulation with local date and time= Mon Sep 19 15:40:17 2022

SysC Enumeration done. Program launching...
Simulation @0 s with local date and time= Mon Sep 19 15:40:17 2022

Simulation @1 ms with local date and time= Mon Sep 19 15:40:32 2022

Simulation @2 ms with local date and time= Mon Sep 19 15:40:43 2022

Simulation @3 ms with local date and time= Mon Sep 19 15:40:53 2022

{"region_name":"example","time_ms":0.46,"ticks":80665}
Simulation @4 ms with local date and time= Mon Sep 19 15:41:03 2022

time (ms) = 0.460943

Info: /OSCI/SystemC: Simulation stopped by user.
Generating saxpy_profile/timing-hooks-saxpy_total_instructions.png
Generating saxpy_profile/timing-hooks-saxpy_total_migrations.png
Generating saxpy_profile/timing-hooks-saxpy.Thread_Enqueue_Map.png
Generating saxpy_prof


        SystemC 2.3.3-Accellera --- Feb 22 2022 09:27:12
        Copyright (c) 1996-2018 by all Contributors,
        ALL RIGHTS RESERVED

real	2m15.931s
user	2m26.679s
sys	0m46.476s


#### View Output

This should have generated the file `saxpy_profile/timing-hooks-saxpy-report.html`. Use the Jupyter file browser to navigate to `saxpy_profile` and open the report in your browser.

### Postcript

Once we've finished our testing, we can clean up some of the logfiles that we used for this example with `make clean`. Uncomment the following line to clean this directory.

In [ ]:
#!make clean